In [1]:
import sqlalchemy
import sqlalchemy.ext.declarative
DB_NAME="sqlalchemy_levelup"

In [2]:
! mysql -e "CREATE DATABASE IF NOT EXISTS sqlalchemy_levelup"

# What is SQLAlchemy?

From [their website](http://www.sqlalchemy.org/):

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper (ORM) that gives application developers the full power and flexibility of SQL

# What is the SQLAlchemy ORM?

The ORM provides a way of modelling the objects of your domain and the relationships between them in Python.

It will persist these objects and their relationships for you, using a Relational Database Management System (RDBMS)

# Where does the ORM fit into SQLAlchemy?

The ORM is built on top of SQLAlchemy's lower-level **Expression Language** API. This API is for working directly with database constructs and expressions in Python (e.g. it contains a `Select` class for building `select` queries)

It is possible, and sometimes preferable, to use ONLY the Expression Language if you require more granular control or if the overhead of the ORM is undesirable.

Alternatively, you can use a combination of both the ORM and the Expression Language to add more granular controlls to an ORM based application if and when they are needed

# A worked example - user management

Inspired by our Authentication service Thug Lyf

The domain objects are Users. Users will have attributes for their email address and password

# Defining the data model

As mentioned above, the ORM looks after the persistence of our domain objects.

Essentially this means we will create a Python class representing a `User`. Instances of this class will have attributes for email address and password. The ORM will store a representation of instantiated `User` objects as records in a "users" table in the database (e.g. MySQL, PostgreSQL).

The classes we add for our domain objects are referred to as *models* or *model classes*.

## Declarative Base Class

To have the ORM look after the persistence of our domain objects, we define our model classes as subclasses of a base class from SQLAlchemy.

This base class is provided by SQLAlchemy through the `declarative_base` class factory (method that returns a Python class)

It is common to have just one `Base` class in your application. You often see it defined in the module scope, allowing other modules to import it.

In [3]:
# Create a declarative base
Base = sqlalchemy.ext.declarative.declarative_base()

## The `User` Model

In [4]:
# Note the distinction between the singular "User" classname and the "users" table name
# This is because an instance of this class is a "user" but the table can contain many users
class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)
    
    def __repr__(self):
        return "User(email={})".format(self.email)

You might have noticed that the "type" of the columns here is either `Integer` or `String`, where elsewhere you may have seen specific `VARCHAR`, `CHAR`, `BIGINT` etc. The difference here is that `Integer` and `String` are generic "SQL" types, which will be represented as `VARCHAR`, `INT` etc. in the database. Using these generic types will make the application more independent from the choice of RDBMS.

### Table objects

When we declare a subclass of the declarative base (i.e. a model), the ORM will create a `Table` object and a mapping between the model attributes (id, email, password) and the `Table` columns.

`Table` is the SQLAlchemy abstraction representing RDBMS tables

In [5]:
# The table object associated with User is accessed by the __table__ class attribute
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)

### Metadata

The declarative base has a `Metadata` object attached to it.

`Metadata` contains a registry of all the `Table` objects created when model classes are declared

We can access this registry under the `tables` attribute of the `metadata` object

In [6]:
Base.metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('email', String(length=255), table=<users>, nullable=False), Column('password', String(length=255), table=<users>, nullable=False), schema=None)})

The registry `Metadata` maintains will be useful in a minute when we want to create actual RDBMS tables

# Connecting to the database

The `Engine` is the source of database connectivity in SQLAlchemy. 

All SQLAlchemy applications will call upon an `Engine` instance at some point in order to create a `Connection` object. This is done through the ```engine.connect()``` method.

Note that this means that SQLAlchemy does not establish an actual connection with the database until it is requested 

In [7]:
database_url = sqlalchemy.engine.url.URL(drivername='mysql', database=DB_NAME,
                                         query={'read_default_file': '~/.my.cnf'})

In [8]:
database_url

mysql:///sqlalchemy_levelup?read_default_file=~/.my.cnf

In [9]:
engine = sqlalchemy.engine.create_engine(database_url, echo=True)  # echo=True logs SQL queries to stdout

## Creating tables

Now we have an engine to provide us with database connectivity, we can use the `Metadata` object to create database tables

In [10]:
Base.metadata.create_all(engine)

2017-01-04 17:49:08,856 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-01-04 17:49:08,857 INFO sqlalchemy.engine.base.Engine ()
2017-01-04 17:49:08,860 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-01-04 17:49:08,861 INFO sqlalchemy.engine.base.Engine ()
2017-01-04 17:49:08,862 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-01-04 17:49:08,863 INFO sqlalchemy.engine.base.Engine ()
2017-01-04 17:49:08,865 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-01-04 17:49:08,866 INFO sqlalchemy.engine.base.Engine ()
2017-01-04 17:49:08,867 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-01-04 17:49:08,868 INFO sqlalchemy.engine.base.Engine ()
2017-01-04 17:49:08,869 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-01-04 17

# A Session with the ORM

Now that we have models, tables and an engine, we are ready to use the ORM to create and store `User` objects.

The ORM abstraction for managing synchronisation with the database is the **session**.

In [11]:
Session = sqlalchemy.orm.sessionmaker(bind=engine)

First, we create a `Session` class, configured to use the `Engine` object we created when it requires a database connection. `Session` objects will not ask for connections until they need them.

Now we can create a session:

In [12]:
session = Session()

And add our first user object

In [15]:
sam = User(email="sam@growthintel.com", password="password")
print sam

session.add(sam)

User(email=sam@growthintel.com)


The session can be thought of as a collection of objects. You can iterate over the session to access the objects within:

In [24]:
[obj for obj in session]

[User(email=sam@growthintel.com)]

## Persistance

At the moment, our object still has not been persisted to the database. It is within the session the the "new" state

In [26]:
session.new

IdentitySet([User(email=sam@growthintel.com)])

- can be queried
- persistance
- adding other objects

## Querying for objects

## Session State

In [14]:
# Code

# Extending the data model - Organisations

Our "customers" at GrowthIntel are businesses, not individual customers. Because of this, our Auth service groups Users into larger units, which we call Organisations.

Organisations can have many Users. Users belong to one and only one Organisation (1 to many relationship)

## Diagram

< TODO >

# Many-to-Many relationships - User Roles

# Testing Applications with SQLAlchemy

Transactions and rollbacks